In [0]:
%run "../includes/configuration"

In [0]:
race_results = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
from pyspark.sql.functions import sum, desc, when, count, col

constructor_standings = race_results \
    .groupBy('race_year', 'team') \
    .agg(sum('points').alias('Points'), count(when(col('position') == 1, True)).alias('Wins')) \
    .orderBy(desc('Wins'))

In [0]:
display(constructor_standings)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('Points'), desc('Wins'))
final_df = constructor_standings \
    .withColumn('rank', rank().over(constructor_rank_spec))
display(final_df)

In [0]:
final_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/constructor_standings")